# 🧠 GenAI Travel Planner: Personalized Itinerary Generator

Welcome to the **GenAI Travel Planner**, a project developed as part of the [Gen AI Intensive Course Capstone 2025Q1](https://www.kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1).

In this notebook, we build a smart itinerary planner that:
- Generates **personalized travel plans** based on user preferences
- Uses real-world datasets of **museums**, **restaurants**, **natural attractions**, and **transit stations**
- Leverages **Generative AI capabilities** to produce structured and creative multi-day itineraries

### 🚀 GenAI Capabilities Demonstrated:
- **Structured Output / JSON Mode** — for day-by-day travel itineraries
- **Few-shot Prompting** — to guide the itinerary generation
- **Retrieval Augmented Generation (RAG)** — we extract real-world POIs from datasets and use them as context

Let’s get planning!

## 📁 Step 1: Load and Preview Datasets

We begin by loading four public datasets related to places of interest:
- **Museums Dataset**: Cultural institutions from the IMLS dataset
- **Yelp Restaurants Dataset**: Dining places with location and category
- **Valley Metro Stations**: Public transport stations in the Phoenix area
- **GNIS Natural Features**: Parks, lakes, trails, etc. from GNIS

These will be used as input for our retrieval and GenAI itinerary generation.

In [1]:
# Dataset sources:
# - Museums: https://www.kaggle.com/datasets/imls/museum-directory/data

import pandas as pd

# Load datasets
museums_df = pd.read_csv("/kaggle/input/museum-directory/museums.csv")  # From IMLS Kaggle dataset
yelp_df = pd.read_csv("/kaggle/input/yelp-restaurants/yelp_restaurants.csv")  # Custom-cleaned subset from Yelp Academic
valley_metro_df = pd.read_csv("/kaggle/input/phoenix-valley-metro-rail-stations/ValleyMetroRailStations.csv")  # Phoenix Light Rail station dataset
gnis_df = pd.read_csv("/kaggle/input/gnisnational/gnis.csv", delimiter="|")  # USGS GNIS dataset (pipe-delimited)

# Preview first 5 rows of each dataset
print("📌 Museums:")
display(museums_df.head())

print("📌 Yelp Restaurants:")
display(yelp_df.head())

print("📌 Valley Metro Rail Stations:")
display(valley_metro_df.head())

print("📌 GNIS Natural Features:")
display(gnis_df.head())

<ipython-input-1-c38e2051f589>:7: DtypeWarning: Columns (9,14,21) have mixed types. Specify dtype option on import or set low_memory=False.
  museums_df = pd.read_csv("/kaggle/input/museum-directory/museums.csv")  # From IMLS Kaggle dataset


📌 Museums:


,Museum ID,Museum Name,Legal Name,Alternate Name,Museum Type,Institution Name,Street Address (Administrative Location),City (Administrative Location),State (Administrative Location),Zip Code (Administrative Location),...,Latitude,Longitude,Locale Code (NCES),County Code (FIPS),State Code (FIPS),Region Code (AAM),Employer ID Number,Tax Period,Income,Revenue
0,8400200098,ALASKA AVIATION HERITAGE MUSEUM,ALASKA AVIATION HERITAGE MUSEUM,NaN,HISTORY MUSEUM,NaN,4721 AIRCRAFT DR,ANCHORAGE,AK,99502,...,61.17925,-149.97254,1.0,20.0,2.0,6,920071852,201312.0,602912.0,550236.0
1,8400200117,ALASKA BOTANICAL GARDEN,ALASKA BOTANICAL GARDEN INC,NaN,"ARBORETUM, BOTANICAL GARDEN, OR NATURE CENTER",NaN,4601 CAMPBELL AIRSTRIP RD,ANCHORAGE,AK,99507,...,61.16890,-149.76708,4.0,20.0,2.0,6,920115504,201312.0,1379576.0,1323742.0
2,8400200153,ALASKA CHALLENGER CENTER FOR SPACE SCIENCE TEC...,ALASKA CHALLENGER CENTER FOR SPACE SCIENCE TEC...,NaN,SCIENCE & TECHNOLOGY MUSEUM OR PLANETARIUM,NaN,9711 KENAI SPUR HWY,KENAI,AK,99611,...,60.56149,-151.21598,3.0,122.0,2.0,6,921761906,201312.0,740030.0,729080.0
3,8400200143,ALASKA EDUCATORS HISTORICAL SOCIETY,ALASKA EDUCATORS HISTORICAL SOCIETY,NaN,HISTORIC PRESERVATION,NaN,214 BIRCH STREET,KENAI,AK,99611,...,60.56280,-151.26597,3.0,122.0,2.0,6,920165178,201412.0,0.0,0.0
4,8400200027,ALASKA HERITAGE MUSEUM,ALASKA AVIATION HERITAGE MUSEUM,NaN,HISTORY MUSEUM,NaN,301 W NORTHERN LIGHTS BLVD,ANCHORAGE,AK,99503,...,61.17925,-149.97254,1.0,20.0,2.0,6,920071852,201312.0,602912.0,550236.0


📌 Yelp Restaurants:


,address,name,stars,longitude,postal_code,business_id,latitude,review_count,categories,city,is_open,hours,attributes,state
0,30 Eglinton Avenue W,Emerald Chinese Restaurant,2.5,-79.652289,L5R 3E7,b'QXAEGFB4oINsVuTFxEYKFQ',43.605499,128,Specialty Food|Restaurants|Dim Sum|Imported Fo...,Mississauga,1,Monday@9:0-0:0|Tuesday@9:0-0:0|Wednesday@9:0-0...,"RestaurantsReservations@True, GoodForMeal@dess...",ON
1,10110 Johnston Rd Ste 15,Musashi Japanese Restaurant,4.0,-80.859132,28210,b'gnKjwL_1w79qoiV3IC_xQQ',35.092564,170,Sushi Bars|Restaurants|Japanese,Charlotte,1,Monday@17:30-21:30|Wednesday@17:30-21:30|Thurs...,"GoodForKids@True, NoiseLevel@uaverage, Restaur...",NC
2,2450 E Indian School Rd,Taco Bell,3.0,-112.028588,85016,b'1Dfx3zM-rW4n-31KeC8sJg',33.495194,18,Restaurants|Breakfast & Brunch|Mexican|Tacos|T...,Phoenix,1,Monday@7:0-0:0|Tuesday@7:0-0:0|Wednesday@7:0-0...,"RestaurantsTakeOut@True, BusinessParking@garag...",AZ
3,5981 Andrews Rd,Marcos Pizza,4.0,-81.359556,44060,b'fweCYi8FmbJXHCqLnwuk8w',41.708520,16,Italian|Restaurants|Pizza|Chicken Wings,Mentor-on-the-Lake,1,Monday@10:0-0:0|Tuesday@10:0-0:0|Wednesday@10:...,"RestaurantsPriceRange2@2, BusinessAcceptsCredi...",OH
4,1775 E Tropicana Ave Ste 29,Carluccios Tivoli Gardens,4.0,-115.128529,89119,b'PZ-LZzSlhSe9utkQYU8pFg',36.100016,40,Restaurants|Italian,Las Vegas,0,NaN,"OutdoorSeating@False, BusinessAcceptsCreditCar...",NV


📌 Valley Metro Rail Stations:


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,OBJECTID,StationId,StationName,Jurisdiction,StationStatus,POINT_X,POINT_Y,Image,Image2,TVM,...,BikeRacks,WaterFountain,Restroom,InfoDisplay,Em_Call,NextRide,Address,ServiceType,x,y
0,1,1,19th Ave / Dunlap,Phoenix,In Operation,-112.099389,33.567090,https://farm8.staticflickr.com/7090/2653039729...,https://farm8.staticflickr.com/7641/2703965492...,NaN,...,NaN,NaN,NaN,NaN,NaN,18608.0,1935 W Dunlap Ave,Light Rail,643925.836614,934023.015092
1,2,35,Center / Main St,Mesa,In Operation,-111.830660,33.415098,https://farm6.staticflickr.com/5812/2398469262...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18602.0,26 East Main Street,Light Rail,726247.568241,878572.861549
2,3,2,Northern / 19th Ave,Phoenix,In Operation,-112.099360,33.553190,https://farm8.staticflickr.com/7505/2710105440...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18607.0,7832 N 19th Ave,Light Rail,644260.340223,928445.860236
3,4,3,Glendale / 19th Ave,Phoenix,In Operation,-112.099329,33.538643,https://farm8.staticflickr.com/7167/2653039369...,https://farm8.staticflickr.com/7377/2686011248...,NaN,...,NaN,NaN,NaN,NaN,NaN,18606.0,6813 N 19th Ave,Light Rail,644228.998688,923101.968832
4,5,21,44th St / Washington,Phoenix,In Operation,-111.987983,33.448170,https://farm6.staticflickr.com/5771/2343906011...,https://farm2.staticflickr.com/1502/2344359052...,NaN,...,NaN,NaN,NaN,NaN,NaN,10018.0,4203 East Washington Street,Light Rail,678247.401575,890599.341535


📌 GNIS Natural Features:


,"FEATURE_ID,FEATURE_NAME,FEATURE_CLASS,STATE_ALPHA,STATE_NUMERIC,COUNTY_NAME,COUNTY_NUMERIC,PRIMARY_LAT_DMS,PRIM_LONG_DMS,PRIM_LAT_DEC,PRIM_LONG_DEC,SOURCE_LAT_DMS,SOURCE_LONG_DMS,SOURCE_LAT_DEC,SOURCE_LONG_DEC,ELEV_IN_M,ELEV_IN_FT,MAP_NAME,DATE_CREATED,DATE_EDITED"
0,"399,Agua Sal Creek,Stream,AZ,4,Apache,1,362740..."
1,"400,Agua Sal Wash,Valley,AZ,4,Apache,1,363246N..."
2,"401,Aguaje Draw,Valley,AZ,4,Apache,1,343439N,1..."
3,"402,Arlington State Wildlife Area,Park,AZ,4,Ma..."
4,"403,Bar X Wash,Stream,AZ,4,Graham,9,322815N,10..."


### 🧹 Step 2: Preprocess the Datasets

We standardize and clean each dataset to ensure they are usable in our GenAI pipeline:

- Museums: Extract name, type, city/state, and location
- Yelp Restaurants: Keep name, cuisine categories, and coordinates
- Valley Metro: Extract station name, location, and address
- GNIS: Extract natural feature names, types, and geo-coordinates

All datasets are filtered to remove entries missing geolocation.